# Cleaning Voting Data

In [2]:
import geopandas as gpd
import mapclassify
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Load data
gdf = gpd.read_file("./va_admin_bndry/VirginiaCounty_ClippedToShoreline.shx") # Load Virginia shapefile for maps
vdf = pd.read_csv('./data/voting_VA.csv') # Load historical voting data
cdf = pd.read_csv('./data/county_adjacencies.csv') # Load county adjacency data and population data

In [3]:
# 2012: 0002_ds191_20125_county_E.csv
# 2016: 0002_ds225_20165_county_E.csv
# 2020: 0002_ds249_20205_county_E.csv
csv_files = ["./data/county_data/0002_ds191_20125_county_E.csv", "./data/county_data/0002_ds225_20165_county_E.csv", "./data/county_data/0002_ds249_20205_county_E.csv"]

identifier_columns = ["STATE", "STATEA", "COUNTYA"]

# relevant columns: 2012 variables, 2016 variables, 2020 variables
# variable order: sex by age, race, ratio of income to poverty level in the past 12 months
# column names pertain to each cattegory of each variable, look at the codebook for more information
relevant_columns = [
    "QSEE001", "QSEE002", "QSEE003", "QSEE004", "QSEE005", "QSEE006", "QSEE007", "QSEE008", "QSEE009", "QSEE010", "QSEE011", "QSEE012", "QSEE013", "QSEE014", "QSEE015", "QSEE016", "QSEE017", "QSEE018", "QSEE019", "QSEE020", "QSEE021", "QSEE022", "QSEE023", "QSEE024", "QSEE025", "QSEE026", "QSEE027", "QSEE028", "QSEE029", "QSEE030", "QSEE031", "QSEE032", "QSEE033", "QSEE034", "QSEE035", "QSEE036", "QSEE037", "QSEE038", "QSEE039", "QSEE040", "QSEE041", "QSEE042", "QSEE043", "QSEE044", "QSEE045", "QSEE046", "QSEE047", "QSEE048", "QSEE049",
    "QSQE001", "QSQE002", "QSQE003", "QSQE004", "QSQE005", "QSQE006", "QSQE007", "QSQE008", "QSQE009", "QSQE010",
    "QUVE001", "QUVE002", "QUVE003", "QUVE004", "QUVE005", "QUVE006", "QUVE007", "QUVE008",
    "AF2AE001", "AF2AE002", "AF2AE003", "AF2AE004", "AF2AE005", "AF2AE006", "AF2AE007", "AF2AE008", "AF2AE009", "AF2AE010", "AF2AE011", "AF2AE012", "AF2AE013", "AF2AE014", "AF2AE015", "AF2AE016", "AF2AE017", "AF2AE018", "AF2AE019", "AF2AE020", "AF2AE021", "AF2AE022", "AF2AE023", "AF2AE024", "AF2AE025", "AF2AE026", "AF2AE027", "AF2AE028", "AF2AE029", "AF2AE030", "AF2AE031", "AF2AE032", "AF2AE033", "AF2AE034", "AF2AE035", "AF2AE036", "AF2AE037", "AF2AE038", "AF2AE039", "AF2AE040", "AF2AE041", "AF2AE042", "AF2AE043", "AF2AE044", "AF2AE045", "AF2AE046", "AF2AE047", "AF2AE048", "AF2AE049",
    "AF2ME001", "AF2ME002", "AF2ME003", "AF2ME004", "AF2ME005", "AF2ME006", "AF2ME007", "AF2ME008", "AF2ME009", "AF2ME010",
    "AF43E001", "AF43E002", "AF43E003", "AF43E004", "AF43E005", "AF43E006", "AF43E007", "AF43E008",
    "AMPKE001", "AMPKE002", "AMPKE003", "AMPKE004", "AMPKE005", "AMPKE006", "AMPKE007", "AMPKE008", "AMPKE009", "AMPKE010", "AMPKE011", "AMPKE012", "AMPKE013", "AMPKE014", "AMPKE015", "AMPKE016", "AMPKE017", "AMPKE018", "AMPKE019", "AMPKE020", "AMPKE021", "AMPKE022", "AMPKE023", "AMPKE024", "AMPKE025", "AMPKE026", "AMPKE027", "AMPKE028", "AMPKE029", "AMPKE030", "AMPKE031", "AMPKE032", "AMPKE033", "AMPKE034", "AMPKE035", "AMPKE036", "AMPKE037", "AMPKE038", "AMPKE039", "AMPKE040", "AMPKE041", "AMPKE042", "AMPKE043", "AMPKE044", "AMPKE045", "AMPKE046", "AMPKE047", "AMPKE048", "AMPKE049",
    "AMPWE001", "AMPWE002", "AMPWE003", "AMPWE004", "AMPWE005", "AMPWE006", "AMPWE007", "AMPWE008", "AMPWE009", "AMPWE010",
    "AMZME001", "AMZME002", "AMZME003", "AMZME004", "AMZME005", "AMZME006", "AMZME007", "AMZME008"
]

#parse through csv files and put into dataFrames list
first_time = True
dataframes = []
for file in csv_files:
    df1 = pd.read_csv(file, low_memory=False, encoding='latin1')
    if first_time:
        df2 = df1[identifier_columns]
        dataframes.append(df2)
        first_time = False
    df1.drop(columns=[col for col in df1.columns if col not in relevant_columns], inplace=True, errors='ignore')
    dataframes.append(df1)

#concat all DataFrames into one DataFrame
demographics = pd.concat(dataframes, axis=1)
demographics = demographics.loc[demographics["STATE"] == "Virginia"]
#find out how many missing values we have from nghis data
print(f"number of missing values: {demographics.isna().sum().sum()}")
#taking a look at the nghis data
print(demographics.head())
print(demographics.shape)

number of missing values: 0
         STATE STATEA COUNTYA QSEE001 QSEE002 QSEE003 QSEE004 QSEE005 QSEE006  \
2821  Virginia     51     001   33454   16230    1001    1085     967     647   
2822  Virginia     51     003   99484   47564    2913    2862    3245    1832   
2823  Virginia     51     005   16297    7956     410     504     532     353   
2824  Virginia     51     007   12674    6272     316     392     442     331   
2825  Virginia     51     009   32301   15406     983     808    1117     624   

     QSEE007  ... AMPWE009 AMPWE010 AMZME001 AMZME002 AMZME003 AMZME004  \
2821     350  ...      104     1047    54387     2189     2931     1712   
2822    2474  ...      511      440    32227     2450     3923     2549   
2823     195  ...      835     2775   101147     3988     3765     1755   
2824     185  ...       13      228    14744     1378      937      617   
2825     500  ...       99      205    12873      517      865      203   

     AMZME005 AMZME006 AMZME007 AM

In [4]:
# merge our selected variables into the county data
demographics["FIPS"] = (demographics["STATEA"].astype(str) + demographics["COUNTYA"].astype(str)).astype(int)
demographics.drop(columns=["STATE", "STATEA", "COUNTYA"], inplace=True, errors='ignore')
county_demographics = pd.merge(cdf, demographics, on="FIPS")
print(county_demographics.head())
print(county_demographics.shape)
county_demographics.to_csv("./data/county_demographics.csv")

           County  Population2022   FIPS  District                   N1  \
0        Accomack           33191  51001         2       Northumberland   
1       Albemarle          114534  51003         5  CharlottesvilleCity   
2  AlexandriaCity          155525  51510         8              Fairfax   
3       Alleghany           14835  51005         6                Craig   
4          Amelia           13455  51007         5         PrinceEdward   

           N2         N3             N4           N5        N6  ... AMPWE009  \
0   Lancaster  Middlesex        Mathews  Northampton       NaN  ...      104   
1      Nelson    Augusta     Rockingham       Greene    Orange  ...      511   
2   Arlington        NaN            NaN          NaN       NaN  ...     1230   
3        Bath  Botetourt  CovingtonCity   Rockbridge       NaN  ...      835   
4  Cumberland   Powhatan   Chesterfield    Dinwiddie  Nottoway  ...       13   

  AMPWE010 AMZME001 AMZME002 AMZME003 AMZME004 AMZME005 AMZME006 AMZ

Combining geodata:

In [7]:
gdf['FIPS_left'] = pd.to_numeric(gdf['STCOFIPS']) 
df = gdf.merge(county_demographics,left_on='FIPS_left',right_on='FIPS') 

print(f"number of missing values: {df.isna().sum().sum()}")
#these missing values come from spliting counties into smaller regions, they are not variables we are going to use for our models and analysis

#datasize
print(df.head())
print(df.shape)

number of missing values: 936
  STCOFIPS     GNIS       NAME          NAMELSAD GSOURCE LADOPT  LASTUPDATE  \
0    51001  1480091   Accomack   Accomack County       T      N  2014-08-20   
1    51003  1675170  Albemarle  Albemarle County       L      Y  2021-03-29   
2    51005  1492459  Alleghany  Alleghany County       V      N  2014-09-15   
3    51007  1497770     Amelia     Amelia County       T      Y  2021-03-29   
4    51009  1480095    Amherst    Amherst County       V      N  2020-09-11   

  JURISTYPE     AREASQMI    Shape_Leng  ...  AMPWE009 AMPWE010  AMZME001  \
0        CO  1195.366291  4.949957e+06  ...       104     1047     54387   
1        CO   725.634334  2.310198e+05  ...       511      440     32227   
2        CO   449.544769  2.062654e+05  ...       835     2775    101147   
3        CO   360.945234  1.776617e+05  ...        13      228     14744   
4        CO   478.766949  1.876610e+05  ...        99      205     12873   

  AMZME002  AMZME003  AMZME004  AMZME0

Combing with voting data from 2020 now:

In [8]:
df20 = vdf.loc[ vdf['year']==2020,:] # Only the 2020 data

Dvotes = df20.loc[(df20['party']=='DEMOCRAT'),:].groupby('county_fips')['candidatevotes'].sum()
Dvotes = Dvotes.rename('dem_votes_2020')
df = df.merge(Dvotes,left_on='FIPS_left',right_on='county_fips') #df1.merge(df2, left_on='lkey', right_on='rkey

Rvotes = df20.loc[(df20['party']=='REPUBLICAN'),:].groupby('county_fips')['candidatevotes'].sum()
Rvotes = Rvotes.rename('rep_votes_2020')
df = df.merge(Rvotes,left_on='FIPS_left',right_on='county_fips') #df1.merge(df2, left_on='lkey', right_on='rkey

print(f"number of missing values: {df.isna().sum().sum()}")
#datasize
print(df.head())
print(df.shape)

number of missing values: 936
  STCOFIPS     GNIS       NAME          NAMELSAD GSOURCE LADOPT  LASTUPDATE  \
0    51001  1480091   Accomack   Accomack County       T      N  2014-08-20   
1    51003  1675170  Albemarle  Albemarle County       L      Y  2021-03-29   
2    51005  1492459  Alleghany  Alleghany County       V      N  2014-09-15   
3    51007  1497770     Amelia     Amelia County       T      Y  2021-03-29   
4    51009  1480095    Amherst    Amherst County       V      N  2020-09-11   

  JURISTYPE     AREASQMI    Shape_Leng  ...  AMZME001 AMZME002  AMZME003  \
0        CO  1195.366291  4.949957e+06  ...     54387     2189      2931   
1        CO   725.634334  2.310198e+05  ...     32227     2450      3923   
2        CO   449.544769  2.062654e+05  ...    101147     3988      3765   
3        CO   360.945234  1.776617e+05  ...     14744     1378       937   
4        CO   478.766949  1.876610e+05  ...     12873      517       865   

  AMZME004  AMZME005  AMZME006  AMZME0

2016 votiing data:

In [9]:
df16 = vdf.loc[ vdf['year']==2016,:] # Only the 2016 data

Dvotes = df16.loc[(df16['party']=='DEMOCRAT'),:].groupby('county_fips')['candidatevotes'].sum()
Dvotes = Dvotes.rename('dem_votes_2016')
df = df.merge(Dvotes,left_on='FIPS_left',right_on='county_fips') #df1.merge(df2, left_on='lkey', right_on='rkey

Rvotes = df16.loc[(df16['party']=='REPUBLICAN'),:].groupby('county_fips')['candidatevotes'].sum()
Rvotes = Rvotes.rename('rep_votes_2016')
df = df.merge(Rvotes,left_on='FIPS_left',right_on='county_fips') #df1.merge(df2, left_on='lkey', right_on='rkey

print(f"number of missing values: {df.isna().sum().sum()}")
#datasize
print(df.head())
print(df.shape)

number of missing values: 936
  STCOFIPS     GNIS       NAME          NAMELSAD GSOURCE LADOPT  LASTUPDATE  \
0    51001  1480091   Accomack   Accomack County       T      N  2014-08-20   
1    51003  1675170  Albemarle  Albemarle County       L      Y  2021-03-29   
2    51005  1492459  Alleghany  Alleghany County       V      N  2014-09-15   
3    51007  1497770     Amelia     Amelia County       T      Y  2021-03-29   
4    51009  1480095    Amherst    Amherst County       V      N  2020-09-11   

  JURISTYPE     AREASQMI    Shape_Leng  ...  AMZME003 AMZME004  AMZME005  \
0        CO  1195.366291  4.949957e+06  ...      2931     1712      2347   
1        CO   725.634334  2.310198e+05  ...      3923     2549      1484   
2        CO   449.544769  2.062654e+05  ...      3765     1755      2497   
3        CO   360.945234  1.776617e+05  ...       937      617       539   
4        CO   478.766949  1.876610e+05  ...       865      203      1140   

  AMZME006  AMZME007  AMZME008  dem_vo

2012 data:

In [10]:
df12 = vdf.loc[ vdf['year']==2012,:] # Only the 2012 data

Dvotes = df12.loc[(df12['party']=='DEMOCRAT'),:].groupby('county_fips')['candidatevotes'].sum()
Dvotes = Dvotes.rename('dem_votes_2012')
df = df.merge(Dvotes,left_on='FIPS_left',right_on='county_fips') #df1.merge(df2, left_on='lkey', right_on='rkey

Rvotes = df12.loc[(df12['party']=='REPUBLICAN'),:].groupby('county_fips')['candidatevotes'].sum()
Rvotes = Rvotes.rename('rep_votes_2012')
df = df.merge(Rvotes,left_on='FIPS_left',right_on='county_fips') #df1.merge(df2, left_on='lkey', right_on='rkey

print(f"number of missing values: {df.isna().sum().sum()}")
#datasize
print(df.head())
print(df.shape)

number of missing values: 936
  STCOFIPS     GNIS       NAME          NAMELSAD GSOURCE LADOPT  LASTUPDATE  \
0    51001  1480091   Accomack   Accomack County       T      N  2014-08-20   
1    51003  1675170  Albemarle  Albemarle County       L      Y  2021-03-29   
2    51005  1492459  Alleghany  Alleghany County       V      N  2014-09-15   
3    51007  1497770     Amelia     Amelia County       T      Y  2021-03-29   
4    51009  1480095    Amherst    Amherst County       V      N  2020-09-11   

  JURISTYPE     AREASQMI    Shape_Leng  ...  AMZME005 AMZME006  AMZME007  \
0        CO  1195.366291  4.949957e+06  ...      2347     2827      1735   
1        CO   725.634334  2.310198e+05  ...      1484     2554      1230   
2        CO   449.544769  2.062654e+05  ...      2497     4580      1763   
3        CO   360.945234  1.776617e+05  ...       539      973       507   
4        CO   478.766949  1.876610e+05  ...      1140      971       246   

  AMZME008  dem_votes_2020  rep_votes_

In [12]:
#export to csv
df.to_csv('./data/full_va_dataset.csv', index=False)